In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
cd drive/My Drive/mln_project

In [0]:
!pip install node2vec

In [0]:
import pandas as pd
import numpy as np
import networkx as nx
import math
from scipy import spatial
import pickle
from sklearn.metrics import accuracy_score
import seaborn as sns
from node2vec import Node2Vec
import random

In [0]:
file2 = 'RegularSeasonCompactResults.csv'
df = pd.read_csv(file2, delimiter = ',', header = None)
df = np.array(df)

In [0]:
print(df[:5])
print(df.shape)

In [0]:
train_graph = nx.DiGraph()
for i in range(1, df.shape[0] - 2000):
  if int(df[i][0]) >= 2015:
    train_graph.add_edge(int(df[i][2]), int(df[i][4]))

In [0]:
players_win_map = {}
for i in train_graph.nodes():
  players_win_map[i] = []
print(len(players_win_map))
print(players_win_map)
  

In [0]:
node2vec = Node2Vec(train_graph, dimensions = 20, walk_length = 16, num_walks = 50)
model = node2vec.fit(window=7, min_count=1)

In [0]:
edges = {}
player_pair_history = {}
train_data = []
train_label = []
for i in range(1, df.shape[0] - 2000):
  if int(df[i][0]) < 2015:
    continue
  ind = random.randint(0, 10)
  wid = int(df[i][2])
  lid = int(df[i][4])
  if ind < 5:
    wh = int(df[i][2])
    bl = int(df[i][4])
  else:
    wh = int(df[i][4])
    bl = int(df[i][2])
  features = list(model[str(wh)] - model[str(bl)])
  emb1 = model[str(wh)]
  emb2 = model[str(bl)]
  # x = np.multiply(emb1, emb2)
  # print(x)
 
  train_data.append(features)
  if int(df[i][2]) == wh:
    train_label.append(1)
  else:
    train_label.append(0)
  edges[(wh, bl)] = 1
  
  

In [0]:
print(len(train_data))
print(len(train_label))

In [0]:
# nodes = list(train_graph.nodes())
# print(nodes)
# false_edge = []
# false_edge_labels = []
# for i in range(len(nodes)):
#   for j in range(i + 1, len(nodes)):
#     # print(nodes[i], nodes[j])
#     if (nodes[i], nodes[j]) not in edges:
      
#       emb1 = model[str(nodes[i])]
#       emb2 = model[str(nodes[j])]
#       x = np.dot(emb1, emb2.T)
#       # print(x)
#       false_edge.append(x)
      
#       false_edge_labels.append(0)

In [0]:
# print(len(false_edge))
# print(len(false_edge_labels))

In [0]:
# import random
# x_sub, y_sub = zip(*random.sample(list(zip(false_edge[:40000], false_edge_labels[:40000])), 8723))
# print(len(x_sub))
# print(len(y_sub))
# for i in x_sub:
#   train_data.append(i)
# for i in y_sub:
#   train_label.append(i)
# print(len(train_data))
# print(len(train_label))

In [0]:
test_data = []
test_label = []
for i in range(df.shape[0] - 2000, df.shape[0]):
  if int(df[i][0]) < 2015:
    continue
  wid = int(df[i][2])
  lid = int(df[i][4])
  ind = random.randint(0, 10)
  if ind < 5:
    wh = int(df[i][2])
    bl = int(df[i][4])
  else:
    wh = int(df[i][4])
    bl = int(df[i][2])
  
  features = list(model[str(wh)] - model[str(bl)])
  emb1 = model[str(wh)]
  emb2 = model[str(bl)]
  # x = np.multiply(emb1, emb2)
  # x = np.sum(emb1, emb2)

  

  test_data.append(features)
  if int(df[i][2]) == wh:
    test_label.append(1)
  else:
    test_label.append(0)
  # print(x)
  

In [0]:
# x_sub, y_sub = zip(*random.sample(list(zip(false_edge[40000:], false_edge_labels[40000:])), 2000))
# print(len(x_sub))
# print(len(y_sub))
# for i in x_sub:
#   test_data.append(i)
# for i in y_sub:
#   test_label.append(i)
# print(len(test_data))
# print(len(test_label))

In [0]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth = 10, random_state=0)
clf.fit(train_data, train_label)
print(clf.score(train_data, train_label))
test_pred = clf.predict(test_data)
print(accuracy_score(test_pred, test_label))
predictions = clf.predict_proba(test_data)
print(roc_auc_score(test_label, predictions[:,1], average = "macro"))
print("Precision = {}".format(precision_score(test_label, test_pred, average='weighted')))

In [0]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300).fit(train_data, train_label)
print(clf.score(train_data, train_label))
predictions = clf.predict_proba(test_data)
test_pred = clf.predict(test_data)
print(accuracy_score(test_pred, test_label))
print(roc_auc_score(test_label, predictions[:, 1], average = "macro"))
print("Precision = {}".format(precision_score(test_label, test_pred, average='weighted')))

In [0]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_data, train_label)
predictions = lr.predict_proba(test_data)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(test_label, predictions[:,1], average = "macro", multi_class='ovr'))
from sklearn.metrics import accuracy_score
test_pred = lr.predict(test_data)
print(accuracy_score(test_pred, test_label))
from sklearn.metrics import precision_score
print("Precision = {}".format(precision_score(test_label, test_pred, average='weighted')))

In [0]:
import xgboost as xgb
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 20 
train_d = xgb.DMatrix(train_data, label = train_label)
test_d = xgb.DMatrix(test_data, label= test_label)
model = xgb.train(param, train_d, steps)
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(test_d)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(test_label, best_preds, average='weighted')))
print("Recall = {}".format(recall_score(test_label, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(test_label, best_preds)))
print(roc_auc_score(test_label, preds[:,1], average = "macro", multi_class='ovr'))